In [0]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = ""

In [0]:
from NN import *
from utils import get_tf_session
from data_utils import make_batch, single_np_datapoint_generator

import tensorflow as tf
import matplotlib.pyplot as plt
import time

In [0]:
train_dataset_files = [
 './data/tfrecords/0.tfrecord',
 './data/tfrecords/1.tfrecord',
 './data/tfrecords/2.tfrecord',
 './data/tfrecords/3.tfrecord',
 './data/tfrecords/4.tfrecord',
 './data/tfrecords/5.tfrecord',
 './data/tfrecords/6.tfrecord',
 './data/tfrecords/7.tfrecord',
 './data/tfrecords/8.tfrecord',
]

validation_dataset_files = ['./data/tfrecords/9.tfrecord']

In [4]:
X_batch, Y_batch = make_batch(train_dataset_files, 200)
loss_weights = tf.placeholder(dtype=tf.float32, shape=(6,))
no_of_filter_fns=[
#     lambda i: 0 if i == 0 else 11 + 3 * (i - 1), 
    lambda i: 10 + 3 * i
]
train_op, loss = create_yolo_model_graph(X_batch, Y_batch, no_of_filter_fns, loss_weights)
X = tf.placeholder(dtype=tf.float32, shape=(None, None, None, 3))
layers, _, pred = get_yolo_NN(X, no_of_filter_fns)
chk_loc = './models/model_1.ckpt'

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [5]:
sess, saver = get_tf_session(chk_loc)
if tf.test.is_gpu_available():
    print("Tensorflow is able to detect gpu...")

No Checkpoint exits to restore!
Tensorflow is able to detect gpu...


In [0]:
avg_loss = 0
print_int = 200
save_int = 1000
st = time.time()
for i in range(5000):
    _, loss_value= sess.run([train_op, loss], feed_dict={loss_weights: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]})
    avg_loss += loss_value
    if i>0 and i%print_int == 0:
        print("Batch No.", i, "Avg loss:", avg_loss/print_int)
        avg_loss = 0
    if i>0 and i%save_int == 0:
        print("Model saved in path: %s" % saver.save(sess, chk_loc))
print("Model saved in path: %s" % saver.save(sess, chk_loc))
print("training took total", time.time() - st, "seconds")

In [0]:
print(calculate_yolo_metric(sess, X, pred, validation_dataset_files))

In [0]:
gen = single_np_datapoint_generator(validation_dataset_files)

In [0]:
img, out = next(gen)
see_and_compare_yolo_outputs(sess, X, pred, img, out)

In [0]:
tf.saved_model.simple_save(sess, './models/yolo_saved_model', 
                           inputs={"x": X}, 
                           outputs={"pred": pred})